In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
class Inception(nn.Module):
    
    def __init__(self, in_planes, kernel_1_x, 
                 kernel_3_in, kernel_3_x, kernel_5_in, kernel_5_x, pool_planes):
        super(Inception, self).__init__()
        
        ############### 1x1 conv brance ########################
        # nn,Conv2d(in_dim, out_Dim, filter) stride=1 padding=1
        self.b1 =nn.Sequential(
        nn.Conv2d(in_planes, kernel_1_x, kernel_size=1),
        nn.BatchNorm2d(kernel_1_x),
        nn.ReLU(True),
        )
        ############### 1x1 conv => 3x3 conv brance ########################
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes,kernel_3_in,kernel_size=1),
            nn.BatchNorm2d(kernel_3_in),
            nn.ReLU(True),
            nn.Conv2d(kernel_3_in,kernel_3_x,kernel_size=3,padding=1),
            nn.BatchNorm2d(kernel_3_x),
            nn.ReLU(True),
        )
        ############### 1x1 conv => 3x3 conv => 3x3 conv brance ############
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, kernel_5_in,kernel_size=1),
            nn.BatchNorm2d(kernel_5_in),
            nn.ReLU(True),
            nn.Conv2d(kernel_5_in,kernel_5_x,kernel_size=3,padding=1),
            nn.BatchNorm2d(kernel_5_x),
            nn.ReLU(True),
            nn.Conv2d(kernel_5_x,kernel_5_x,kernel_size=3,padding=1),
            nn.BatchNorm2d(kernel_5_x),
            nn.ReLU(True),
        )
        self.b4=nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )
    def forward(self,x):
        y1=self.b1(x) #2 64 227 227
        y2=self.b2(x) #2 128 227 227 => y1+y2= 2 192 227 227
        y3=self.b3(x) #2 32 227 227 => y1+y2+y3 = 2 224 227 227
        y4=self.b4(x) #2 32 227 227 => 2 256 227 227
        return torch.cat([y1,y2,y3,y4],1)

In [3]:
def dimension_check():
    net=Inception(1, 8, 24, 32, 8, 16, 16)
    x= torch.randn(2,1,640,480)
    y=net(x)
    print(y.size())

In [4]:
dimension_check()

torch.Size([2, 72, 640, 480])


In [5]:
class GoogleNet(nn.Module):
    def __init__(self):
        super(GoogleNet,self).__init__()
        self.pre_layers= nn.Sequential(
            nn.Conv2d(1,8,kernel_size=3,padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
        )
        self.a3= Inception(8, 16, 24, 32, 8, 16, 16) # 16 32 16 16 = 60
        self.b3 = Inception(80, 32, 32, 64, 16, 32, 32) # 32 64 32 32 = 160
        
        self.max_pool = nn.MaxPool2d(3,stride=2,padding=1)
        self.a4 = Inception(160, 48,  48, 80, 32,  32,  32)# 48 80 32 32 = 192
        self.b4 = Inception(192, 60, 60, 96, 48,  48,  48) # 60 96 48 48 = 252
        self.c4 = Inception(252, 60, 64, 96, 48,  48,  48) # 60 96 48 48 
        self.d4 = Inception(252, 60, 96, 128, 48,  60,  60) # 308
        self.e4 = Inception(308, 72, 128, 256, 60, 72, 72) # 472
        self.a5 = Inception(472, 72, 128, 256, 60, 72, 72)
        self.b5 = Inception(472, 90, 128, 384, 48, 128, 128)
        
        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.linear = nn.Linear(2190, 7)
        
    def forward(self,x):
        x= self.pre_layers(x)
        x= self.a3(x)       # 2 80 320 240
        x=self.max_pool(x)
        x= self.b3(x)       # 2 160 320 120
        x= self.max_pool(x) # 2 160 160 120
        x = self.a4(x)      # 2 192 160 120
        x=self.max_pool(x)  # 2 192 80 60
        x = self.b4(x)      # 2 512 24 24
        x=self.max_pool(x)  # 252 40 30
        x = self.c4(x)      # 2 512 24 24
        x=self.max_pool(x) #252 20 15
        x = self.d4(x)      # 2 528 24 24
        x = self.e4(x)      # 2 832 24 24
        x = self.max_pool(x)# 2 832 12 12
        x = self.a5(x)      # 2 832 12 12
        x = self.b5(x)      # 2 1024 12 12
        x = self.avgpool(x) # 2 1024 6 6
        x = x.view(x.size(0), -1) # 36864
        x = self.linear(x)  # 10
        return x

In [6]:
def dimension_check():
    net=GoogleNet()
    x= torch.randn(1,1,640,480)
    y=net(x)
    print(y.size())

In [7]:
dimension_check()

torch.Size([1, 7])


In [8]:
input_data=np.load("CK+dataset.npy")
label=np.load("CK+label.npy")
label=label-1

In [9]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [10]:
remake=input_data.unsqueeze(0)
remake=remake.transpose(0,1)

In [11]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(remake,label)

In [12]:
trainset,valset=torch.utils.data.random_split(dataset,[277,50])
trainset,testset=torch.utils.data.random_split(trainset,[227,50])

partition={'train':trainset,'val':valset,'test':testset}

In [13]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=8,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [19]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=64, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [20]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=8, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [21]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [22]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=GoogleNet()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

  0%|          | 0/29 [00:00<?, ?it/s]

time : 8.05, train_loss : 2.10, train_acc : 20.26,val_acc : 12.00,val_loss : 1.99


  0%|          | 0/29 [00:00<?, ?it/s]

time : 8.05, train_loss : 1.91, train_acc : 23.79,val_acc : 30.00,val_loss : 1.62


  0%|          | 0/29 [00:00<?, ?it/s]

...

time : 8.40, train_loss : 0.30, train_acc : 93.39,val_acc : 50.00,val_loss : 2.78


  0%|          | 0/1 [00:00<?, ?it/s]

time : 8.35, train_loss : 0.15, train_acc : 93.39,val_acc : 60.00,val_loss : 1.68


  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 4.58 GiB (GPU 0; 10.76 GiB total capacity; 5.13 GiB already allocated; 3.59 GiB free; 6.07 GiB reserved in total by PyTorch)